In [0]:
import pandas as pd
from pathlib import Path

VOLUME_NAME = "basedatospnc-lab8"  
BASE_DIR = Path(f"/Volumes/workspace/default/{VOLUME_NAME}")
INPUT_XLSX = BASE_DIR / "Bases de datos principales PNC.xlsx"  
OUT_DIR = BASE_DIR / "RawData"
OUT_DIR.mkdir(parents=True, exist_ok=True)
SHEETS = [s for s in (f"cuadro {i}" for i in range(1, 66))]

In [0]:
xls = pd.ExcelFile(INPUT_XLSX)
for sheet in SHEETS:
    if sheet not in xls.sheet_names:
        print(f"[WARN] La hoja '{sheet}' no existe en el archivo. Se omite.")
        continue

    df = pd.read_excel(xls, sheet_name=sheet, dtype=object)  
    out_csv = OUT_DIR / f"{sheet.lower()}.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] Exportada hoja '{sheet}' -> {out_csv}")

[OK] Exportada hoja 'cuadro 1' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 1.csv
[OK] Exportada hoja 'cuadro 2' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 2.csv
[OK] Exportada hoja 'cuadro 3' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 3.csv
[OK] Exportada hoja 'cuadro 4' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 4.csv
[OK] Exportada hoja 'cuadro 5' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 5.csv
[OK] Exportada hoja 'cuadro 6' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 6.csv
[OK] Exportada hoja 'cuadro 7' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 7.csv
[OK] Exportada hoja 'cuadro 8' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 8.csv
[OK] Exportada hoja 'cuadro 9' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 9.csv
[OK] Exportada hoja 'cuadro 10' -> /Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 10.cs

Mostrar cuántos registros hay en cada tabla (hechos, vehículos, fallecidos, lesionados). Muestre algunos resultados con la función .show(). Genere un describe y summary para aquellas columnas que considere importantes según cada archivo

In [0]:
import re, random

random.seed(42)
files = {f.path.split("/")[-1].lower(): f.path for f in dbutils.fs.ls(str(OUT_DIR))}

def find_csv_for(i: int):
    candidates = [
        f"cuadro {i}.csv",
        f"cuadro_{i}.csv",
        f"Cuadro {i}.csv".lower(),
        f"Cuadro_{i}.csv".lower(),
    ]
    for c in candidates:
        if c.lower() in files:
            return files[c.lower()]
    return None

# --- Función para limpiar nombres de columnas ---
def normalizar_cols(df):
    nuevos = []
    vistos = set()
    for i, c in enumerate(df.columns):
        name = str(c) if c is not None else f"col_{i+1}"
        # quitar espacios adelante/atrás
        name = name.strip()
        # reemplazar espacios por "_"
        name = re.sub(r"\s+", "_", name)
        # quitar caracteres no alfanuméricos
        name = re.sub(r"[^0-9A-Za-z_]", "", name)
        # evitar nombres vacíos
        if not name:
            name = f"col_{i+1}"
        # asegurar que no se repitan nombres
        base = name
        k = 1
        while name in vistos:
            k += 1
            name = f"{base}_{k}"
        vistos.add(name)
        nuevos.append(name)
    return df.toDF(*nuevos)

# 2) Cargar todos los que existan del 1 al 65
dfs = {}         
counts = {}
loaded_names = [] 

for i in range(1, 66):
    name = f"cuadro {i}"
    path = find_csv_for(i)
    if not path:
        print(f"[WARN] No encontré CSV para {name} en {OUT_DIR}")
        continue
    try:
        df = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(path))
        # normalizar columnas
        df = normalizar_cols(df)
        dfs[name] = df
        cnt = df.count()
        counts[name] = cnt
        loaded_names.append(name)
        print(f"[OK] Cargado {name}: {cnt} filas, {len(df.columns)} columnas -> {path}")
    except Exception as e:
        print(f"[ERR] No se pudo cargar {name} desde {path}: {e}")

if not loaded_names:
    displayHTML("<p style='color:#c00'>No se cargó ningún cuadro. Revisa los nombres de archivos en OUT_DIR.</p>")

# 3) Elegir 5 al azar (o menos si hay menos de 5 cargados)
k = min(5, len(loaded_names))
sampled = random.sample(loaded_names, k) if k > 0 else []

# 4) Mostrar 5 tablas aleatorias con título + conteo + vista de 10 filas
for name in sampled:
    df = dfs[name]
    n  = counts[name]
    displayHTML(f"<h3 style='margin:10px 0'>{name.title()}</h3>"
                f"<p style='margin:0 0 8px 0'>Registros: <b>{n}</b></p>")
    display(df.limit(10))

[OK] Cargado cuadro 1: 26 filas, 6 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 1.csv
[OK] Cargado cuadro 2: 23 filas, 14 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 2.csv
[OK] Cargado cuadro 3: 23 filas, 9 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 3.csv
[OK] Cargado cuadro 4: 13 filas, 6 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 4.csv
[OK] Cargado cuadro 5: 13 filas, 9 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 5.csv
[OK] Cargado cuadro 6: 8 filas, 13 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 6.csv
[OK] Cargado cuadro 7: 26 filas, 9 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 7.csv
[OK] Cargado cuadro 8: 23 filas, 11 columnas -> dbfs:/Volumes/workspace/default/basedatospnc-lab8/RawData/cuadro 8.csv
[OK] Cargado cuadro 9: 13 filas, 11 columnas -> dbfs:/

Cuadro 15 Registros: 24

Zona_de_ocurrencia,Total,Lunes,Martes,Mircoles,Jueves,Viernes,Sbado,Domingo
Total,1739,218,202,184,200,279,353,303
1,182,21,29,20,24,28,26,34
2,37,5,6,5,3,6,7,5
3,51,5,6,9,6,6,8,11
4,27,3,3,1,5,7,6,2
5,69,12,6,8,4,11,13,15
6,96,15,14,15,10,10,17,15
7,219,24,18,21,19,37,56,44
8,25,3,6,2,2,0,8,4
9,90,9,15,10,10,20,13,13


Cuadro 4 Registros: 13

Mes_de_ocurrencia,2020,2021,2022,2023,2024
Total,6350,8153,7924,8218,8401
Enero,610,695,714,696,626
Febrero,670,626,614,683,711
Marzo,590,657,727,635,780
Abril,329,644,632,938,630
Mayo,374,701,671,696,674
Junio,372,625,554,647,549
Julio,405,614,683,640,703
Agosto,571,643,624,605,687
Septiembre,495,648,612,615,686


...

Tipo_de_vehculo,Total,Hombre,Mujer,Ignorado,total_colision,hombre_colision,mujer_colision,ignorado_colision,total_atropello,hombre_atropello,mujer_atropello,ognorado_atropello,total_derrape,hombre_derrape,mujer_derrape,choque_total,choque_hombre,choque_mujer,choque_ignorado,total_vuelco,hombre_vuelco,mujer_vuelco,total_embarranc,hombre_embarranc,mujer_embarranc,ignorado_embarranc,total_encunet,hombre_encunet,mujeres_encunet,total_caida,mujer_caida,ignorado_caida,hombre_ignorado,mujer_ignorado
Total,8907,6446,2445,16,5289,3931,1355,3,1495,986,499,10,1027,820,207,321,216,103,2,341,199,142,262,164,97,1,6,5,1,1,1,165,125,40
Motocicleta,5675,4389,1280,6,3865,3061,801,3,673,405,266,2,952,780,172,81,68,13,0,2,2,0,31,27,3,1,0,0,0,0,0,71,46,25
Automóvil,952,650,296,6,376,232,144,0,379,279,94,6,9,9,0,63,35,28,0,34,26,8,49,30,19,0,1,0,1,0,0,41,39,2
Pick Up,675,407,268,0,351,217,134,0,101,69,32,0,23,6,17,52,30,22,0,97,51,46,27,17,10,0,5,5,0,0,0,19,12,7
Camioneta,397,245,150,2,215,131,84,0,103,66,37,0,0,0,0,24,14,8,2,14,8,6,31,19,12,0,0,0,0,0,0,10,7,3
Bus Urbano,313,159,154,0,73,30,43,0,38,23,15,0,4,1,3,27,17,10,0,98,53,45,71,34,37,0,0,0,0,1,1,1,1,0
Camión,260,199,60,1,115,93,22,0,48,31,16,1,2,2,0,37,28,9,0,39,30,9,11,10,1,0,0,0,0,0,0,8,5,3
Microbús,229,123,106,0,133,70,63,0,14,7,7,0,6,5,1,18,10,8,0,29,10,19,28,20,8,0,0,0,0,0,0,1,1,0
Moto Taxi,137,68,69,0,87,40,47,0,14,11,3,0,5,2,3,5,2,3,0,18,10,8,7,2,5,0,0,0,0,0,0,1,1,0
Cabezal,63,52,11,0,30,25,5,0,23,17,6,0,1,1,0,1,1,0,0,7,7,0,1,1,0,0,0,0,0,0,0,0,0,0


Cuadro 32 Registros: 23

Departamento,Total,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
Total,8907,671,721,840,686,717,526,672,700,737,832,893,912
Guatemala,3516,277,292,315,278,268,228,286,269,260,320,357,366
El Progreso,214,12,11,25,20,14,21,13,20,13,21,22,22
Sacatepéquez,384,34,24,26,31,13,19,35,39,57,38,30,38
Chimaltenango,215,14,24,15,9,22,7,10,16,17,27,37,17
Escuintla,1253,78,109,159,107,103,61,74,105,101,98,130,128
Santa Rosa,242,28,23,30,28,20,11,9,14,18,23,9,29
Sololá,100,13,1,4,7,7,2,9,15,6,6,8,22
Totonicapán,88,4,1,14,1,5,3,11,8,7,17,10,7
Quetzaltenango,240,15,16,17,12,12,20,26,17,25,18,46,16


Cuadro 29 Registros: 23

Departamento,2020,2021,2022,2023,2024
Total,8142,10544,10722,11198,11274
Guatemala,3025,3982,3947,4074,4092
El Progreso,198,253,265,276,276
Sacatepéquez,262,336,466,468,478
Chimaltenango,316,372,403,372,349
Escuintla,659,896,1041,1322,1549
Santa Rosa,283,333,380,442,338
Sololá,101,177,93,139,126
Totonicapán,74,79,113,107,102
Quetzaltenango,240,296,364,392,311


In [0]:
# ======================= MUESTRA DINÁMICA (reemplaza las 4 llamadas fijas) =======================
TITULOS = {
    1: "Accidentes de tránsito ocurridos en la República de Guatemala, por año, según departamento. Serie histórica 2020 - 2024.",
    2: "Accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024.",
    3: "Accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    4: "Accidentes de tránsito ocurridos en la República de Guatemala, por año, según mes. Serie histórica 2020 - 2024.",
    5: "Accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según mes, año 2024.",
    6: "Accidentes de tránsito ocurridos en la República de Guatemala, por año, según día de la semana. Serie histórica 2020 - 2024.",
    7: "Accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según hora de ocurrencia, año 2024.",
    8: "Accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según departamento, año 2024.",
    9: "Accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según mes, año 2024.",
    10: "Accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según día de ocurrencia, año 2024.",
    11: "Accidentes de tránsito ocurridos por atropello en la República de Guatemala, por mes, según departamento, año 2024.",
    12: "Accidentes de tránsito ocurridos por atropello en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    13: "Accidentes de tránsito ocurridos por atropello en la República de Guatemala, por día de la semana, según mes, año 2024.",
    14: "Accidentes de tránsito ocurridos en el municipio de Guatemala, por zona de ocurrencia, según hora, año 2024.",
    15: "Accidentes de tránsito ocurridos en el municipio de Guatemala, por día de la semana, según zona de ocurrencia, año 2024.",
    16: "Accidentes de tránsito ocurridos en el municipio de Guatemala, por tipo de accidente, según zona de ocurrencia, año 2024.",
    17: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por año, según departamento. Serie histórica 2020 -2024.",
    18: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según tipo de vehículo, año 2024.",
    19: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según color de vehículo, año 2024.",
    20: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según modelo de vehículo, año 2024.",
    21: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por tipo de vehículo, según departamento, año 2024.",
    22: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y condición del conductor, según departamento, año 2024.",
    23: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y condición del conductor, según mes, año 2024.",
    24: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según condición del conductor y sexo, año 2024.",
    25: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y condición del conductor, según hora, año 2024.",
    26: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y condición del conductor, según grupos de edad, año 2024",
    27: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en la República de Guatemala, por tipo de accidente, según tipo de vehículo y sexo, año 2024.",
    28: "Cantidad de vehículos involucrados en accidentes de tránsito ocurridos en el municipio de Guatemala, por tipo de vehículo, según zona de ocurrencia, año 2024.",
    29: "Víctimas por accidentes de tránsito ocurridos en la República de Guatemala por año, según departamento. Serie histórica 2020 - 2024.",
    30: "Víctimas por accidentes de tránsito ocurridos en la República de Guatemala, por sexo y estado del implicado, según departamento, año 2024.",
    31: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por año, según departamento. Serie histórica 2020 - 2024.",
    32: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024.",
    33: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    34: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por grupos de edad, según departamento, año 2024.",
    35: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según hora, año 2024.",
    36: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y tipo de accidente, según tipo de vehículo, año 2024.",
    37: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según tipo de accidente y sexo, año 2024.",
    38: "Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo, según grupos de edad, año 2024.",
    39: "Lesionados por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024.",
    40: "Lesionados por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    41: "Lesionados por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por grupos de edad, según departamento, año 2024.",
    42: "Lesionados por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según hora, año 2024.",
    43: "Lesionados por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por sexo, según tipo de vehículo, año 2024.",
    44: "Lesionados en accidentes de tránsito ocurridos en el municipio de Guatemala, por grupos de edad, según zona de ocurrencia, año 2024.",
    45: "Lesionados en accidentes de tránsito ocurridos en el municipio de Guatemala, por día de la semana, según zona de ocurrencia, año 2024.",
    46: "Lesionados en accidentes de tránsito ocurridos en el municipio de Guatemala, por sexo, según zona de ocurrencia, año 2024.",
    47: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por año, según departamento. Serie histórica 2020 - 2024.",
    48: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024.",
    49: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    50: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por grupos de edad, según departamento, año 2024.",
    51: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según hora, año 2024.",
    52: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y tipo de accidente, según tipo de vehículo, año 2024.",
    53: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según tipo de accidente y sexo, año 2024.",
    54: "Fallecidos en accidentes de tránsito ocurridos en la República de Guatemala, por sexo, según grupos de edad, año 2024.",
    55: "Fallecidos por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024.",
    56: "Fallecidos por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según departamento, año 2024.",
    57: "Fallecidos por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por grupos de edad, según departamento, año 2024.",
    58: "Fallecidos por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por día de la semana, según hora, año 2024.",
    59: "Fallecidos por atropello en accidentes de tránsito ocurridos en la República de Guatemala, por sexo, según tipo de vehículo, año 2024.",
    60: "Fallecidos en accidentes de tránsito ocurridos en el municipio de Guatemala, por grupos de edad, según zona de ocurrencia, año 2024.",
    61: "Fallecidos en accidentes de tránsito ocurridos en el municipio de Guatemala, por día de la semana, según zona de ocurrencia, año 2024.",
    62: "Fallecidos en accidentes de tránsito ocurridos en el municipio de Guatemala, por sexo, según zona de ocurrencia, año 2024.",
    63: "Tasa de víctimas involucradas en accidentes de tránsito ocurridos en la República de Guatemala por año, según departamento por cada 100,000 habitantes. Serie histórica 2020 - 2024.",
    64: "Tasa de víctimas lesionadas involucradas en accidentes de tránsito ocurridos en la República de Guatemala por año, según departamento por cada 100,000 habitantes. Serie histórica 2020 – 2024.",
    65: "Tasa de víctimas fallecidas involucradas en accidentes de tránsito ocurridos en la República de Guatemala por año, según departamento por cada 100,000 habitantes. Serie histórica 2020 - 2024."
}

import re

def _titulo_para(name: str) -> str:
    # name viene como "cuadro 12", extraemos el índice para buscar en TITULOS
    m = re.search(r"(\d+)$", name.strip())
    if m:
        idx = int(m.group(1))
        return TITULOS.get(idx, f"Cuadro {idx}")
    return name.title()

from pyspark.sql import functions as F

def show_sample(df, titulo, n=10, cols=None, truncate=False):
    displayHTML(f"<h4 style='margin:6px 0'>{titulo} — muestra de {n} filas</h4>")
    # Si nos pasan columnas, filtramos a las que existan
    if cols:
        if isinstance(cols, str):
            cols = [c.strip() for c in cols.split(",")]
        keep = [c for c in cols if c in df.columns]
        if keep:
            df = df.select(*[F.col(c) for c in keep])
    # En Databricks, display(df.limit(n)) es más cómodo que df.show()
    display(df.limit(n))

# Qué mostramos: si ya tienes 'sampled', úsalo; si no, usa hasta 5 de loaded_names
a_mostrar = sampled if sampled else loaded_names[:5]

for name in a_mostrar:
    df = dfs[name]
    titulo = _titulo_para(name)
    show_sample(df, titulo, n=10)
# ================================================================================================

Accidentes de tránsito ocurridos en el municipio de Guatemala, por día de la semana, según zona de ocurrencia, año 2024. — muestra de 10 filas

Zona_de_ocurrencia,Total,Lunes,Martes,Mircoles,Jueves,Viernes,Sbado,Domingo
Total,1739,218,202,184,200,279,353,303
1,182,21,29,20,24,28,26,34
2,37,5,6,5,3,6,7,5
3,51,5,6,9,6,6,8,11
4,27,3,3,1,5,7,6,2
5,69,12,6,8,4,11,13,15
6,96,15,14,15,10,10,17,15
7,219,24,18,21,19,37,56,44
8,25,3,6,2,2,0,8,4
9,90,9,15,10,10,20,13,13


Accidentes de tránsito ocurridos en la República de Guatemala, por año, según mes. Serie histórica 2020 - 2024. — muestra de 10 filas

Mes_de_ocurrencia,2020,2021,2022,2023,2024
Total,6350,8153,7924,8218,8401
Enero,610,695,714,696,626
Febrero,670,626,614,683,711
Marzo,590,657,727,635,780
Abril,329,644,632,938,630
Mayo,374,701,671,696,674
Junio,372,625,554,647,549
Julio,405,614,683,640,703
Agosto,571,643,624,605,687
Septiembre,495,648,612,615,686


Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por sexo y tipo de accidente, según tipo de vehículo, año 2024. — muestra de 10 filas

Tipo_de_vehculo,Total,Hombre,Mujer,Ignorado,total_colision,hombre_colision,mujer_colision,ignorado_colision,total_atropello,hombre_atropello,mujer_atropello,ognorado_atropello,total_derrape,hombre_derrape,mujer_derrape,choque_total,choque_hombre,choque_mujer,choque_ignorado,total_vuelco,hombre_vuelco,mujer_vuelco,total_embarranc,hombre_embarranc,mujer_embarranc,ignorado_embarranc,total_encunet,hombre_encunet,mujeres_encunet,total_caida,mujer_caida,ignorado_caida,hombre_ignorado,mujer_ignorado
Total,8907,6446,2445,16,5289,3931,1355,3,1495,986,499,10,1027,820,207,321,216,103,2,341,199,142,262,164,97,1,6,5,1,1,1,165,125,40
Motocicleta,5675,4389,1280,6,3865,3061,801,3,673,405,266,2,952,780,172,81,68,13,0,2,2,0,31,27,3,1,0,0,0,0,0,71,46,25
Automóvil,952,650,296,6,376,232,144,0,379,279,94,6,9,9,0,63,35,28,0,34,26,8,49,30,19,0,1,0,1,0,0,41,39,2
Pick Up,675,407,268,0,351,217,134,0,101,69,32,0,23,6,17,52,30,22,0,97,51,46,27,17,10,0,5,5,0,0,0,19,12,7
Camioneta,397,245,150,2,215,131,84,0,103,66,37,0,0,0,0,24,14,8,2,14,8,6,31,19,12,0,0,0,0,0,0,10,7,3
Bus Urbano,313,159,154,0,73,30,43,0,38,23,15,0,4,1,3,27,17,10,0,98,53,45,71,34,37,0,0,0,0,1,1,1,1,0
Camión,260,199,60,1,115,93,22,0,48,31,16,1,2,2,0,37,28,9,0,39,30,9,11,10,1,0,0,0,0,0,0,8,5,3
Microbús,229,123,106,0,133,70,63,0,14,7,7,0,6,5,1,18,10,8,0,29,10,19,28,20,8,0,0,0,0,0,0,1,1,0
Moto Taxi,137,68,69,0,87,40,47,0,14,11,3,0,5,2,3,5,2,3,0,18,10,8,7,2,5,0,0,0,0,0,0,1,1,0
Cabezal,63,52,11,0,30,25,5,0,23,17,6,0,1,1,0,1,1,0,0,7,7,0,1,1,0,0,0,0,0,0,0,0,0,0


Lesionados en accidentes de tránsito ocurridos en la República de Guatemala, por mes, según departamento, año 2024. — muestra de 10 filas

Departamento,Total,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
Total,8907,671,721,840,686,717,526,672,700,737,832,893,912
Guatemala,3516,277,292,315,278,268,228,286,269,260,320,357,366
El Progreso,214,12,11,25,20,14,21,13,20,13,21,22,22
Sacatepéquez,384,34,24,26,31,13,19,35,39,57,38,30,38
Chimaltenango,215,14,24,15,9,22,7,10,16,17,27,37,17
Escuintla,1253,78,109,159,107,103,61,74,105,101,98,130,128
Santa Rosa,242,28,23,30,28,20,11,9,14,18,23,9,29
Sololá,100,13,1,4,7,7,2,9,15,6,6,8,22
Totonicapán,88,4,1,14,1,5,3,11,8,7,17,10,7
Quetzaltenango,240,15,16,17,12,12,20,26,17,25,18,46,16


Víctimas por accidentes de tránsito ocurridos en la República de Guatemala por año, según departamento. Serie histórica 2020 - 2024. — muestra de 10 filas

Departamento,2020,2021,2022,2023,2024
Total,8142,10544,10722,11198,11274
Guatemala,3025,3982,3947,4074,4092
El Progreso,198,253,265,276,276
Sacatepéquez,262,336,466,468,478
Chimaltenango,316,372,403,372,349
Escuintla,659,896,1041,1322,1549
Santa Rosa,283,333,380,442,338
Sololá,101,177,93,139,126
Totonicapán,74,79,113,107,102
Quetzaltenango,240,296,364,392,311


In [0]:
import re

def _idx(name: str) -> int:
    m = re.search(r"(\d+)$", name.strip())
    return int(m.group(1)) if m else 10**9

numeric_primitives = ("byte", "short", "int", "bigint", "long", "float", "double")

for name in sorted(loaded_names, key=_idx):
    df = dfs[name]

    # Detect numeric columns (including decimal with any precision/scale)
    num_cols = [
        c for c, t in df.dtypes
        if t in numeric_primitives or t.lower().startswith("decimal")
    ]

    displayHTML(f"<h4 style='margin:10px 0'>{name.title()} — columnas numéricas</h4>")
    if not num_cols:
        displayHTML("<p style='margin:0 0 8px 0'><i>Sin columnas numéricas detectadas.</i></p>")
        continue

    # Ensure all columns exist in DataFrame
    existing_num_cols = [c for c in num_cols if c in df.columns]
    if not existing_num_cols:
        displayHTML("<p style='margin:0 0 8px 0'><i>No valid numeric columns found in DataFrame.</i></p>")
        continue

    # Show statistical summary of numeric columns
    displayHTML("<p style='margin:6px 0 4px 0'><i>Resumen (describe):</i></p>")
    display(
        df.select(*existing_num_cols).describe()
    )

Cuadro 1 — columnas numéricas

Resumen (describe):

summary,2020,2021,2022,2023,2024
count,23,23,23,23,23
mean,552.1739130434783,708.9565217391304,689.0434782608696,714.6086956521739,730.5217391304348
stddev,1369.43431095318,1759.7049462974512,1714.6234274684678,1777.0637871380172,1809.9987661865516
min,49,77,68,40,49
max,6350,8153,7924,8218,8401


Cuadro 2 — columnas numéricas

Resumen (describe):

...

Resumen (describe):

summary,2020,2021,2022,2023,2024
count,23,23,23,23,23
mean,37.899007661813904,45.98942857921133,47.22729776990445,47.96487284064589,47.40683564275011
stddev,19.208482678174025,23.63176988414674,28.033293408208166,30.97813530014346,34.688566852202506
min,10.49810596671517,9.933062591871897,11.564096633796598,10.316233831882526,8.238227943664025
max,75.62063303792945,97.47298202399217,102.72239910462864,125.53357841113827,150.235124564462


Cuadro 65 — columnas numéricas

Resumen (describe):

summary,2020,2021,2022,2023,2024
count,23,23,23,23,23
mean,10.409124161861136,14.272388487543198,14.378673318520585,14.95047408640869,15.011883499471567
stddev,6.709878873542623,9.159590613673977,8.237840149658068,10.210806031216327,9.294429807348218
min,2.114201896074583,3.7838210538530226,2.803417365768872,2.957320365139658,2.7197351755006256
max,26.933376150495416,35.63956190753901,32.196572853689574,40.11842552953783,35.49050029615384


In [0]:
from pyspark.sql import Row

percentiles = [0.25, 0.5, 0.75]
rel_tol = 0.01  # tolerancia de approxQuantile

for name in sorted(loaded_names, key=_idx):
    df = dfs[name]
    num_cols = [c for c, t in df.dtypes
                if t in numeric_primitives or t.lower().startswith("decimal")]

    if not num_cols:
        continue

    displayHTML(f"<h4 style='margin:10px 0'>{name.title()} — percentiles</h4>")
    rows = []
    for c in num_cols:
        qs = df.approxQuantile(c, percentiles, rel_tol)
        rows.append(Row(col=c, p25=qs[0], p50=qs[1], p75=qs[2]))
    spark.createDataFrame(rows).show(truncate=False)

Cuadro 1 — percentiles

+----+-----+-----+-----+
|col |p25  |p50  |p75  |
+----+-----+-----+-----+
|2020|121.0|197.0|229.0|
|2021|154.0|228.0|270.0|
|2022|138.0|216.0|297.0|
|2023|129.0|198.0|300.0|
|2024|128.0|194.0|299.0|
+----+-----+-----+-----+



Cuadro 2 — percentiles

+----------+-----+-----+-----+
|col       |p25  |p50  |p75  |
+----------+-----+-----+-----+
|Total     |128.0|194.0|299.0|
|Enero     |7.0  |15.0 |25.0 |
|Febrero   |10.0 |17.0 |28.0 |
|Marzo     |14.0 |23.0 |29.0 |
|Abril     |11.0 |14.0 |22.0 |
|Mayo      |11.0 |17.0 |25.0 |
|Junio     |6.0  |13.0 |23.0 |
|Julio     |12.0 |15.0 |24.0 |
|Agosto    |8.0  |17.0 |21.0 |
|Septiembre|9.0  |15.0 |30.0 |
|Octubre   |11.0 |17.0 |33.0 |
|Noviembre |12.0 |19.0 |27.0 |
|Diciembre |12.0 |19.0 |31.0 |
+----------+-----+-----+-----+



Cuadro 3 — percentiles

...

+----+------------------+------------------+-----------------+
|col |p25               |p50               |p75              |
+----+------------------+------------------+-----------------+
|2020|26.773701977461005|46.42430912701917 |67.19482138231545|
|2021|37.322874493927124|58.31828825769086 |75.4520480980239 |
|2022|37.50218099073812 |53.081003191445916|74.67868513108075|
|2023|33.26166912457135 |59.985847089202394|75.74649590597829|
|2024|32.758531701516056|54.088705476982256|73.15716453219456|
+----+------------------+------------------+-----------------+



Cuadro 64 — percentiles

+----+------------------+------------------+------------------+
|col |p25               |p50               |p75               |
+----+------------------+------------------+------------------+
|2020|20.080276483095755|35.189220533984034|55.28205051584129 |
|2021|30.575236167341433|44.898417330789094|60.574179458976936|
|2022|30.045183612270886|43.19357593725152 |58.97304342185187 |
|2023|25.870187096888827|35.59917563164591 |64.11864016774263 |
|2024|25.27989584682911 |36.47773515900976 |53.987968395614686|
+----+------------------+------------------+------------------+



Cuadro 65 — percentiles

+----+------------------+------------------+------------------+
|col |p25               |p50               |p75               |
+----+------------------+------------------+------------------+
|2020|5.494224290793486 |9.136305496237751 |13.855495333958189|
|2021|6.747638326585695 |12.565937565642413|18.367694361565825|
|2022|6.9323264129218565|14.743260855462967|19.16664686641164 |
|2023|7.391482027682522 |12.766389577983782|20.07461882705471 |
|2024|7.478635854686945 |13.425579442295717|22.677815788584855|
+----+------------------+------------------+------------------+



#2. Identificar los años disponibles en cada tabla y validar si coinciden.

In [0]:
# ===================== DETECCIÓN DE AÑOS EN TODAS LAS TABLAS (1–65) =====================
def _idx(name: str) -> int:
    m = re.search(r"(\d+)$", name.strip())
    return int(m.group(1)) if m else 10**9

YEAR_COL_REGEX = re.compile(
    r"^(anio|ano|a[o]?|year|anio_?hecho|ano_?hecho|anio_?ocurrencia|ano_?ocurrencia|anio_?registro)$",
    re.I
)

YEAR_MIN, YEAR_MAX = 1990, 2035

def _years_from_candidate_cols(df):
    years = set()
    candidates = [c for c in df.columns if YEAR_COL_REGEX.search(c)]
    for c in candidates:
        ydf = (df
               .select(F.col(c).cast("int").alias("y"))
               .where(F.col("y").isNotNull() & (F.col("y") >= YEAR_MIN) & (F.col("y") <= YEAR_MAX))
               .select("y").distinct())
        years.update([r["y"] for r in ydf.collect()])
    return years

def _years_from_headers(df):
    """Si la tabla está en formato ancho (años como columnas), detecta encabezados 'YYYY' válidos."""
    years = set()
    for c in df.columns:
        cc = c.strip()
        if re.fullmatch(r"\d{4}", cc):
            y = int(cc)
            if YEAR_MIN <= y <= YEAR_MAX:
                years.add(y)
    return years

def detectar_anios(df):
    """Intenta por columnas de año; si no hay, intenta por encabezados."""
    y1 = _years_from_candidate_cols(df)
    if y1:
        return sorted(y1)
    y2 = _years_from_headers(df)
    if y2:
        return sorted(y2)
    return []

displayHTML("<h3 style='margin:8px 0'>Años disponibles por tabla</h3>")

anios_por_tabla = {}
problemas_fuera_de_rango = []

for name in sorted(loaded_names, key=_idx):
    df = dfs[name]
    anios = detectar_anios(df)
    anios_por_tabla[name] = anios if anios else None
    lista = ", ".join(map(str, anios)) if anios else "<i>No se encontraron años</i>"
    displayHTML(f"<p style='margin:4px 0'><b>{name.title()}:</b> {lista}</p>")

# --- Validación global ---
validas = [set(v) for v in anios_por_tabla.values() if v]

if validas:
    all_equal = all(s == validas[0] for s in validas)
    inter = sorted(list(set.intersection(*validas))) if len(validas) > 1 else sorted(list(validas[0]))
    uni = sorted(list(set.union(*validas)))        if len(validas) > 1 else sorted(list(validas[0]))

    displayHTML("<h3 style='margin:12px 0 4px 0'>Validación</h3>")
    displayHTML(f"<p style='margin:0'><b>¿Todos iguales?</b> {'Sí' if all_equal else 'No'}</p>")
    displayHTML(f"<p style='margin:0'><b>Intersección:</b> {', '.join(map(str, inter)) if inter else '—'}</p>")
    displayHTML(f"<p style='margin:0'><b>Unión:</b> {', '.join(map(str, uni)) if uni else '—'}</p>")
else:
    displayHTML("<p style='margin:8px 0'><i>No hay tablas con años detectados para comparar.</i></p>")
#==============================================================================

Años disponibles por tabla

Cuadro 1: 2020, 2021, 2022, 2023, 2024

Cuadro 2: No se encontraron años

Cuadro 3: No se encontraron años

Cuadro 4: 2020, 2021, 2022, 2023, 2024

...

Cuadro 65: 2020, 2021, 2022, 2023, 2024

Validación

¿Todos iguales? No

Intersección: 2024

Unión: 2020, 2021, 2022, 2023, 2024

# 3. Mostrar los valores distintos de tipo de accidente.

In [0]:
# ===================== TIPOS DE ACCIDENTE (valores distintos) =====================
import re
from pyspark.sql import functions as F

# --- 1) identificar qué cuadros "van de tipo de accidente" a partir del diccionario ---
diccionario = TITULOS  
cuadros_tipo = sorted([i for i, desc in diccionario.items() if "tipo de accidente" in desc.lower()])

# --- 2) normalizador simple de columnas (igual al que usamos antes) ---
import re as _re
def normalizar_cols(df):
    nuevos, vistos = [], set()
    for i, c in enumerate(df.columns):
        name = str(c) if c is not None else f"col_{i+1}"
        name = name.strip()
        name = _re.sub(r"\s+", "_", name)
        name = _re.sub(r"[^0-9A-Za-z_]", "", name)
        if not name:
            name = f"col_{i+1}"
        base, k = name, 1
        while name in vistos:
            k += 1
            name = f"{base}_{k}"
        vistos.add(name)
        nuevos.append(name)
    return df.toDF(*nuevos)

# --- 3) helpers de detección ---
RE_TIPO_COL = re.compile(r"(tipo.*acciden|acciden.*tipo)", re.I)

# columnas de dimensión / medidas a excluir cuando la tabla esté "ancha"
EXCLUDE_HEADERS = set([
    "departamento","depto","municipio","zona","mes","dia","hora",
    "anio","ano","ao","anio_hecho","ano_hecho","anio_ocurrencia","ano_ocurrencia","anio_registro",
    "sexo","condicion","estado","grupos_de_edad","grupo_de_edad","grupo_edad","edad",
    "tipo_de_vehiculo","tipo_vehiculo","vehiculo","color","modelo",
    "total","cantidad","conteo","victimas","lesionados","fallecidos"
])

YEAR_MIN, YEAR_MAX = 1990, 2035

def _is_year_col(name: str) -> bool:
    return bool(re.fullmatch(r"\d{4}", name)) and (YEAR_MIN <= int(name) <= YEAR_MAX)

def _idx(name: str) -> int:
    m = re.search(r"(\d+)$", name.strip())
    return int(m.group(1)) if m else 10**9

# --- 4) recorrer y mostrar valores distintos ---
displayHTML("<h3 style='margin:8px 0'>Valores distintos de <i>tipo de accidente</i></h3>")

tipos_global = set()

for i in cuadros_tipo:
    name = f"cuadro {i}"
    if name not in dfs:
        displayHTML(f"<p><b>{name.title()}:</b> <i>No cargado.</i></p>")
        continue

    df_raw = dfs[name]
    df = normalizar_cols(df_raw)

    # a) intentar columna explícita
    tipo_col = None
    for c in df.columns:
        if RE_TIPO_COL.search(c):
            tipo_col = c
            break

    tipos = []
    if tipo_col:
        # columna encontrada -> extraer valores distintos (string limpio, sin nulos)
        vals = (df.select(F.col(tipo_col).cast("string").alias("tipo"))
                  .where(F.col("tipo").isNotNull() & (F.length(F.trim("tipo")) > 0))
                  .select(F.trim("tipo").alias("tipo")).distinct()
                  .orderBy("tipo"))
        tipos = [r["tipo"] for r in vals.collect()]
    else:
        # b) formato ancho: los encabezados son los tipos
        # candidatos = columnas que NO sean año, NO estén en EXCLUDE_HEADERS y NO sean 4 dígitos
        candidatos = []
        for c in df.columns:
            cl = c.lower()
            if _is_year_col(c):
                continue
            if cl in EXCLUDE_HEADERS:
                continue
            # si parece claramente métrica numérica genérica, saltar (opcional)
            if cl in ("valor","valores","monto","cantidad_total"):
                continue
            candidatos.append(c)

        # si la tabla es bien ancha, estos suelen ser los tipos
        # Validación ligera: si hay muy pocos candidatos y además ninguna col de "tipo", mostramos aviso
        tipos = sorted(candidatos)

    if tipos:
        tipos_global.update(tipos)
        lista = ", ".join(tipos)
        displayHTML(f"<p style='margin:4px 0'><b>{name.title()}:</b> {lista}</p>")
    else:
        displayHTML(f"<p style='margin:4px 0'><b>{name.title()}:</b> <i>No se detectaron tipos</i></p>")

# --- 5) resumen global ---
if tipos_global:
    displayHTML("<h4 style='margin:12px 0 6px 0'>Resumen global (únicos)</h4>")
    displayHTML("<p style='margin:0'>" + ", ".join(sorted(tipos_global)) + "</p>")
else:
    displayHTML("<p style='margin:12px 0'><i>No se detectaron tipos de accidente en los cuadros identificados.</i></p>")
# =============================================================================================

Valores distintos de tipo de accidente

Cuadro 8: Atropello, Cada, Choque, Colisin, Derrape, Embarranc, Encunet, Ignorado, Vuelco

Cuadro 9: Atropello, Cada, Choque, Colisin, Derrape, Embarranc, Encunet, Ignorado, Mes_de_ocurrencia, Vuelco

Cuadro 10: Atropello, Cada, Choque, Colisin, Da_de_ocurrencia, Derrape, Embarranc, Encunet, Ignorado, Vuelco

Cuadro 16: Atropello, Choque, Colisin, Derrape, Embarranc, Vuelco, Zona_de_ocurrencia

...

Cuadro 37: Atropello, Caída, Choque, Colisión, Derrape, Embarrancó, Encunetó, Hombre, Ignorado, Mujer, Total, Vuelco

Cuadro 52: Hombres, Ignorado, Mujeres, Tipo_de_vehculo, hombre_atropello, hombre_choque, hombre_colision, hombre_derrape, hombre_embarranc, hombre_encunet, hombre_ignorado, hombre_vuelco, ignorado_atropello, mujer_atropello, mujer_choque, mujer_colision, mujer_derrape, mujer_embarranc, mujer_ignorado, mujer_vuelco, total_atropello, total_choque, total_colision, total_derrape, total_embarranc, total_encunet, total_ignorado, total_vuelco

Cuadro 53: Atropello, Choque, Colisión, Derrape, Embarrancó, Encunetó, Hombre, Ignorado, Mujer, Total, Vuelco

Resumen global (únicos)

Atropello, Cada, Caída, Choque, Colisin, Colisión, Color_de_vehculo, Da_de_ocurrencia, Derrape, Embarranc, Embarrancó, Encunet, Encunetó, Hombre, Hombres, Ignorado, Mes_de_ocurrencia, Modelo_de_vehculo, Mujer, Mujeres, Tipo_de_vehculo, Tipo_de_vehculo_y_sexo_de_la_persona, Total, Unnamed_11, Unnamed_12, Vuelco, Zona_de_ocurrencia, choque_hombre, choque_ignorado, choque_mujer, choque_total, hombre_atropello, hombre_choque, hombre_colision, hombre_derrape, hombre_embarranc, hombre_encunet, hombre_ignorado, hombre_vuelco, ignorado_atropello, ignorado_caida, ignorado_colision, ignorado_embarranc, mujer_atropello, mujer_caida, mujer_choque, mujer_colision, mujer_derrape, mujer_embarranc, mujer_ignorado, mujer_vuelco, mujeres_encunet, ognorado_atropello, total_atropello, total_caida, total_choque, total_colision, total_derrape, total_embarranc, total_encunet, total_ignorado, total_vuelco

#4. Calcular cuántos departamentos únicos aparecen en las bases.

In [0]:
# ===================== DEPARTAMENTOS ÚNICOS (filtrando por TITULOS) =====================
import re
from pyspark.sql import functions as F

# 1) Identificar qué cuadros mencionan "departamento" en el título/descripción
cuadros_con_departamento = sorted(
    [i for i, txt in TITULOS.items() if re.search(r"departament", txt, re.I)]
)

# 2) Helper: localizar la columna de departamento en cada DF
DEPTO_COL_REGEX = re.compile(r"^(depto|departamento|departament|dpto|dep)$", re.I)

def encontrar_col_departamento(df):
    for c in df.columns:
        if DEPTO_COL_REGEX.search(c.strip()):
            return c
    return None

# 3) Recorrer solo esos cuadros y reunir valores únicos
departamentos_global = set()

for i in cuadros_con_departamento:
    name = f"cuadro {i}"
    if name not in dfs:
        print(f"[WARN] {name} no está cargado; se omite.")
        continue

    df = dfs[name]
    col = encontrar_col_departamento(df)
    if not col:
        print(f"[INFO] {name} no tiene columna de departamento detectable; se omite.")
        continue

    vals = (df.select(F.col(col).cast("string").alias("depto"))
              .where(F.col("depto").isNotNull() & (F.length(F.trim("depto")) > 0))
              .select(F.trim("depto").alias("depto")).distinct())
    departamentos_global.update([r["depto"] for r in vals.collect()])

# 4) Mostrar resultados
lista_departamentos = sorted(departamentos_global)
displayHTML("<h3 style='margin:8px 0'>Departamentos únicos (solo cuadros que mencionan “departamento”)</h3>")
displayHTML(f"<p style='margin:0 0 6px 0'><b>Total:</b> {len(lista_departamentos)}</p>")
displayHTML("<p>" + ", ".join(lista_departamentos) + "</p>")
# =========================================================================================

Departamentos únicos (solo cuadros que mencionan “departamento”)

Total: 23

Alta Verapaz, Baja Verapaz, Chimaltenango, Chiquimula, El Progreso, Escuintla, Guatemala, Huehuetenango, Izabal, Jalapa, Jutiapa, Petén, Quetzaltenango, Quiché, Retalhuleu, Sacatepéquez, San Marcos, Santa Rosa, Sololá, Suchitepéquez, Total, Totonicapán, Zacapa

# 5. ¿Cuál es el total de accidentes por año y departamento? Apóyese de la función groupBy. Investigue la función display que tiene Databricks y muestre su resultado en formato de gráfico de barras.

In [0]:
from pyspark.sql import functions as F
import re

df = dfs["cuadro 1"]

dept_col = next((c for c in df.columns if re.search(r"(depto|depart|departamento)", c, re.I)), df.columns[0])
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", c)]

df_total = df.filter(F.lower(F.col(dept_col)).like("total%"))
if df_total.count() == 0:
    df_total = df.groupBy().agg(*[F.sum(F.col(c)).alias(c) for c in year_cols])

df_total_long = (df_total
    .select(
        F.explode(
            F.arrays_zip(
                F.array(*[F.lit(c) for c in year_cols]),
                F.array(*[F.col(c) for c in year_cols])
            )
        ).alias("kv")
    )
    .select(F.col("kv.0").alias("año"), F.col("kv.1").cast("int").alias("total_accidentes"))
    .orderBy("año"))

display(df_total_long)

año,total_accidentes
2020,6350
2021,8153
2022,7924
2023,8218
2024,8401


Databricks visualization. Run in Databricks to view.

In [0]:
df_dept = df.filter(~F.lower(F.col(dept_col)).like("total%"))

df_long = (df_dept
    .select(
        F.col(dept_col),
        F.explode(
            F.arrays_zip(
                F.array(*[F.lit(c) for c in year_cols]),
                F.array(*[F.col(c) for c in year_cols])
            )
        ).alias("kv")
    )
    .select(
        F.col(dept_col).alias("departamento"),
        F.col("kv.0").alias("anio"),
        F.col("kv.1").cast("int").alias("accidentes")
    ))

df_grouped = (df_long.groupBy("anio", "departamento")
                       .agg(F.sum("accidentes").alias("total_accidentes"))
                       .orderBy("anio", "departamento"))

# Mostrar en Databricks
display(df_grouped)


anio,departamento,total_accidentes
2020,,null
2020,Alta Verapaz,296
2020,Baja Verapaz,96
2020,Chimaltenango,229
2020,Chiquimula,130
2020,El Progreso,124
2020,Escuintla,440
2020,Guatemala,2676
2020,Huehuetenango,113
2020,Izabal,230


Databricks visualization. Run in Databricks to view.

# 6. ¿Qué día de la semana registra más accidentes en 2024? Graficar con display en un gráfico de columnas

In [0]:
from pyspark.sql import functions as F
import re, unicodedata

df = dfs["cuadro 3"]  # ajusta el nombre si difiere

def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s or "") if unicodedata.category(c) != "Mn")

def norm(s: str) -> str:
    s = strip_accents(s).lower()
    return re.sub(r"[^a-z]", "", s)

# Variantes por día (para capturar 'Mircoles', 'Sbado', abreviaturas, etc.)
day_variants = {
    "Lunes":      ["lun"],
    "Martes":     ["mar"],
    "Miercoles":  ["mie","mir"],   # 'mircoles' sin 'e'
    "Jueves":     ["jue"],
    "Viernes":    ["vier","vie"],
    "Sabado":     ["sab","sb"],     # 'sbado' sin 'a'
    "Domingo":    ["dom"],
}

# Detectar columna real por día
colmap = {}
for c in df.columns:
    nk = norm(c)
    for day, roots in day_variants.items():
        if any((nk.startswith(r) or r in nk) for r in roots):
            # Si hay empate, quédate con el match más largo (más específico)
            if day not in colmap or len(nk) > len(norm(colmap[day])):
                colmap[day] = c

faltan = [d for d in day_variants if d not in colmap]
if faltan:
    raise ValueError(f"No encontré todas las columnas de días. Faltan: {faltan}. Detectadas: {colmap}")

# Quitar la fila Total si existe
col_dep = next((c for c in df.columns if "depar" in norm(c)), None)
df_work = df.where(F.lower(F.col(col_dep).cast("string")) != "total") if col_dep else df

# Unpivot a formato largo
pairs = []
ordered_days = ["Lunes","Martes","Miercoles","Jueves","Viernes","Sabado","Domingo"]
for d in ordered_days:
    pairs += [f"'{d}'", f"`{colmap[d]}`"]
stack_expr = f"stack(7, {', '.join(pairs)}) as (dia, accidentes)"

df_long = (df_work.selectExpr(stack_expr)
                   .select("dia", F.col("accidentes").cast("double")))

# Sumar 2024 por día
df_dia = (df_long.groupBy("dia")
                 .agg(F.sum(F.coalesce("accidentes", F.lit(0))).alias("total_accidentes"))
                 .orderBy(F.desc("total_accidentes")))

# Mostrar tabla (en la UI selecciona Bar chart: X=dia, Y=total_accidentes)
display(df_dia)

dia,total_accidentes
Sabado,1615.0
Domingo,1602.0
Viernes,1182.0
Lunes,1141.0
Jueves,981.0
Martes,970.0
Miercoles,910.0


Databricks visualization. Run in Databricks to view.

# 7. Mostrar la distribución de accidentes por hora del día en el municipio de Guatemala. Graficar en un histograma.

In [0]:
from pyspark.sql import functions as F

# 1) Tomamos el cuadro 14
df = dfs["cuadro 14"]

# 2) Quitamos columnas 'Total' e 'Ignorada' si existen
cols_to_drop = [c for c in ["Total", "Ignorada"] if c in df.columns]
df_clean = df.drop(*cols_to_drop)

# 3) (Por si venían como filas) quitamos filas 'Ignorada' y 'Total'
df_clean = df_clean.filter(~F.col("Hora_de_ocurrencia").isin(["Ignorada", "Total"]))

# 4) Sumamos todas las columnas numéricas por franja horaria
num_cols = [c for c in df_clean.columns if c != "Hora_de_ocurrencia"]
df_horas = (
    df_clean
    .withColumn(
        "total_accidentes",
        sum(F.coalesce(F.col(c).cast("double"), F.lit(0.0)) for c in num_cols)
    )
    .select("Hora_de_ocurrencia", "total_accidentes")
)

# 5) Ordenamos por la hora inicial (00, 01, 02, …)
df_horas = (
    df_horas
    .withColumn("start_h", F.regexp_extract("Hora_de_ocurrencia", r"^(\d{2})", 1).cast("int"))
    .orderBy("start_h")
    .drop("start_h")
)

# 6) Mostrar y graficar con display (en la UI: Plot -> Bar -> Keys=Hora_de_ocurrencia, Values=total_accidentes)
display(df_horas)

Hora_de_ocurrencia,total_accidentes
00:00 a 00:59,115.0
01:00 a 01:59,73.0
02:00 a 02:59,76.0
03:00 a 03:59,45.0
04:00 a 04:59,34.0
05:00 a 05:59,55.0
06:00 a 06:59,49.0
07:00 a 07:59,57.0
08:00 a 08:59,35.0
09:00 a 09:59,57.0


Databricks visualization. Run in Databricks to view.